# Random Forest 03
Notebook for lung cancer images, modify for glioma images. After bug fixes, this ran to completion.

Train a default random forest on 6-way cancer classification using just Image.csv data.

Do 4-fold cross-validation, and aggregate patch-level predictions up to the WSI level. We are getting low accuracy here.

For now, aggregate by majority voting, with each patch casting an equal vote. We have a bug that gives aggregate accuracy 0.

In [1]:
import datetime
print(datetime.datetime.now())
from platform import python_version
print('Python',python_version())
import csv
import numpy as np
import pandas as pd
import matplotlib as plt
import random
import sklearn   # pip install --upgrade scikit-learn
print('sklearn',sklearn.__version__)

2022-07-19 12:54:46.162388
Python 3.8.10
sklearn 1.0.2


In [2]:
# Shuffle X,y in tandem -- can cause shape problems
from sklearn.utils import shuffle
# The model
from sklearn.ensemble import RandomForestClassifier

In [3]:
# Our input is the output from the Data notebooks.
# These csv files were filtered to remove give-away columns and bad rows.
BASE_PATH='/home/jrm/Adjeroh/Naved/July_Run/CellProfilerFiltered/'
# This is the patch-level csv file: one row per patch, with nucleus totals.
# Later, incorporate the nucleus-specific csv files.
FILENAME='Process100_Image.csv'
# This directory contains lists of patch filenames,
# divided into 5 folds of train/valid splits.
# Notice patch filename [0:19] is case ID, [0:23] is WSI ID,
# and last column is the 6-way cancer class (zero to five) of the case.
# For example, fold0_train.txt starts:
# TCGA-06-0129-01Z-00-DX1_5400_5100.png, 0
# TCGA-06-0129-01Z-00-DX1_5700_6000.png, 0
LEN_PATIENT_ID = 19
SPLITS_BASE='/home/jrm/Adjeroh/Naved/July_Run/TrainTestSplit/'
SPLITS_FILES = [
    ('fold0_train.txt','fold0_test.txt'),
    ('fold1_train.txt','fold1_test.txt'),
    ('fold2_train.txt','fold2_test.txt'),
    ('fold3_train.txt','fold3_test.txt'),
    ('fold4_train.txt','fold4_test.txt'),
]
NUM_FOLDS = 5
PATIENT_TO_CANCER = {}  # hash case ID to class number

## Data loading functions

In [4]:
# sklearn can do this but we were stumbling on shape & type problems
def list_shuffle(X):   
    L = len(X)
    for i in range(L):
        r = random.randrange(0,L)
        if i!=r:
            temp = X[i]
            X[i] = X[r]
            X[r] = temp
    return X

In [5]:
# Given a case ID = patient name = directory, load the csv file.
# Expect a value like p='TCGA-S9-A6UB-01Z-00'
def load_patient_data(p):
    filepath=BASE_PATH+p+'/'+FILENAME
    df = pd.read_csv(filepath)
    return df

In [6]:
# Given fold, load the case/patient IDs separated into training and validation.
# Fold should be an in between 0 and 4 for 5-fold cross validation.
# The returned list has unique strings like  TCGA-06-0129-01Z-00
def _load_patient_names(filename):
    patients=[]
    with open(filename) as infile:
        rows = csv.reader(infile)
        for row in rows:
            patient = row[0][:LEN_PATIENT_ID]
            cancer_class = row[1]
            if patient in PATIENT_TO_CANCER:
                if PATIENT_TO_CANCER[patient] != cancer_class:
                    raise Exception('One patient in two classes:',patient)
            else:
                PATIENT_TO_CANCER[patient] = cancer_class
            patients.append(patient)
    patients = list(np.unique(patients))
    return patients
def load_patient_names(fold):
    filenames = SPLITS_FILES[fold]  # tuple of train,valid
    train_patients = _load_patient_names(SPLITS_BASE+filenames[0])
    valid_patients = _load_patient_names(SPLITS_BASE+filenames[1])
    return train_patients,valid_patients

In [7]:
# Load csv rows for one set of train+valid patient names.
# This is slow, probably due to concatenating dataframes.
# Consider a rewrite that concatenates csv before constructing a dataframe.
def _load_train_valid_set(patients):
    X = None  # instances = features
    y = None  # labels = cancer class
    z = []  # patient_id for aggregation
    for patient in patients:
        #print('Load patient:',patient)
        df = load_patient_data(patient)
        # apply the same cancer label to all patches from one patient
        label = PATIENT_TO_CANCER[patient]
        if X is None:
            X = df 
            y = np.full(shape=len(df), fill_value=label, dtype=np.int8)
        else:
            X = pd.concat( (X, df) )
            more = np.full(shape=len(df), fill_value=label, dtype=np.int8)
            y = np.concatenate( (y, more) )
        z = z + [patient]*len(df)  # same ID for all patches from one patient
    return X,y,z
def load_train_valid_set(train_patient_names,valid_patient_names):
    print('Load train set...')
    X_train,y_train,z_train = _load_train_valid_set(train_patient_names)
    print('Loaded lengths X,y,z:',X_train.shape,len(y_train),len(z_train))
    print('Load valid set...')
    X_valid,y_valid,z_valid = _load_train_valid_set(valid_patient_names)
    print('Loaded lengths X,y,z:',X_valid.shape,len(y_valid),len(z_valid))
    
    # TO DO: raise exception if any valid patient is also a train patient
    return X_train,y_train,z_train,X_valid,y_valid,z_valid

## Aggregation functions

In [8]:
# Later, weight each patch label by the confidence i.e. score
def aggregate_accuracy(y_pred,z_valid):
    L = len(y_pred)
    if L != len(z_valid):
        raise Exception('Lengths of y and z do not match')
    correct = {}
    incorrect = {}
    patients = np.unique(z_valid)
    for patient in patients:
        correct[patient]=0
        incorrect[patient]=0
    for i in range(L):
        patient = z_valid[i]
        label = PATIENT_TO_CANCER[patient]
        pred = y_pred[i]
        if pred == label:
            correct[patient] += 1
        else:
            incorrect[patient] += 1
    numerator = 0
    denominator = 0
    for patient in patients:
        denominator += 1
        if correct[patient]>incorrect[patient]:
            numerator += 1
        print('Patient',patient,'Correct',correct[patient],'Wrong',incorrect[patient])
    accuracy = float(0)
    if denominator>0:
        accuracy = 100.0*numerator/denominator 
    return accuracy

## Main loop: Load, Classify, Report

In [9]:
patch_accuracies = []
patient_accuracies = []
for fold in range(NUM_FOLDS):
    print()
    print(datetime.datetime.now(),'Fold',fold,'Loading...')
    train_patients,valid_patients = load_patient_names(fold)  
    X_train,y_train,z_train,X_valid,y_valid,z_valid = \
        load_train_valid_set(train_patients,valid_patients)    
    # print('Fold',fold,'patients train',train_patients,'patients valid',valid_patients)
    # print('Fold:',fold,'patches train',X_train.shape,'patches valid',X_valid.shape)

    # This shuffle is pro forma, not strictly necessary.
    # CNN models are sensitive to train set order but RF models are not. 
    print(datetime.datetime.now(),'Shuffle...')
    X_train,y_train = shuffle(X_train,y_train)

    print(datetime.datetime.now(),'Train...')
    rfc = RandomForestClassifier()
    rfc.fit(X_train,y_train)  # slow
    
    print(datetime.datetime.now(),'Validate...')
    y_pred = rfc.predict(X_valid)
    matches = np.count_nonzero(y_valid==y_pred)
    accuracy = 100.0 * matches / len(y_pred)
    patch_accuracies.append(accuracy)
    print('Fold',fold,'Patch-level Validation Accuracy:',accuracy)
    
    accuracy = aggregate_accuracy(y_pred,z_valid)
    patient_accuracies.append(accuracy)
    print('Fold',fold,'Patient-level Validation Accuracy:',accuracy)
    X_train=None
    X_valid=None
    y_train=None
    y_valid=None
    z_train=None
    z_valid=None
    rfc = None
    y_pred = None
    matches = None


2022-07-19 12:54:46.879244 Fold 0 Loading...
Load train set...
Loaded lengths X,y,z: (59992, 5303) 59992 59992
Load valid set...
Loaded lengths X,y,z: (15773, 5303) 15773 15773
2022-07-19 12:56:11.289489 Shuffle...
2022-07-19 12:56:17.926094 Train...
2022-07-19 13:03:25.203407 Validate...
Fold 0 Patch-level Validation Accuracy: 48.63374120332213
Patient TCGA-02-0004-01Z-00 Correct 0 Wrong 385
Patient TCGA-02-0010-01Z-00 Correct 0 Wrong 1563
Patient TCGA-02-0025-01Z-00 Correct 0 Wrong 698
Patient TCGA-02-0338-01Z-00 Correct 0 Wrong 292
Patient TCGA-02-0430-01Z-00 Correct 0 Wrong 395
Patient TCGA-02-0439-01Z-00 Correct 0 Wrong 788
Patient TCGA-06-0185-01Z-00 Correct 0 Wrong 392
Patient TCGA-06-0241-01Z-00 Correct 0 Wrong 383
Patient TCGA-06-0879-01Z-00 Correct 0 Wrong 457
Patient TCGA-28-1746-01Z-00 Correct 0 Wrong 177
Patient TCGA-DB-A4XF-01Z-00 Correct 0 Wrong 802
Patient TCGA-DH-A66B-01Z-00 Correct 0 Wrong 389
Patient TCGA-DU-5874-01Z-00 Correct 0 Wrong 401
Patient TCGA-DU-7009-01Z-0

In [10]:
print(datetime.datetime.now())
print('Cross validation patch-level accuracy:',patch_accuracies)
print('mean:',np.mean(patch_accuracies),'std:',np.std(patch_accuracies))
print('Cross validation patient-level accuracy:',patient_accuracies)
print('mean:',np.mean(patient_accuracies),'std:',np.std(patient_accuracies))


2022-07-19 13:36:45.892009
Cross validation patch-level accuracy: [48.63374120332213, 67.75651114382902, 40.51832329317269, 57.135328219242254, 58.80691371309436]
mean: 54.5701635145321 std: 9.285528627618548
Cross validation patient-level accuracy: [0.0, 0.0, 0.0, 0.0, 0.0]
mean: 0.0 std: 0.0
